<a href="https://colab.research.google.com/github/sab112/myfirstrepo/blob/master/myStockPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [161]:
import quandl
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [162]:
quandl.ApiConfig.api_key='8sea8tfXW2CSAPzEYUoq'
df= quandl.get('XNSE/BAJAJ_AUTO')
df 

,Open,High,Low,Close,Volume,Adjustment Factor,Adjustment Type
Date,,,,,,,
2010-01-01,669.844609,669.844609,669.844609,669.844609,0.0,NaN,NaN
2010-01-04,673.738166,676.772851,658.488399,661.160448,381510.0,NaN,NaN
2010-01-05,667.554280,670.741654,652.743492,664.214219,463938.0,NaN,NaN
2010-01-06,668.012346,668.012346,650.892144,653.945915,465832.0,NaN,NaN
2010-01-07,657.705870,659.996198,639.421418,642.475188,329288.0,NaN,NaN
...,...,...,...,...,...,...,...
2018-12-25,2550.461719,2550.461719,2550.461719,2550.461719,0.0,NaN,NaN
2018-12-26,2540.211758,2574.003867,2499.305522,2559.775610,410582.0,NaN,NaN
2018-12-27,2587.202446,2587.296053,2524.204970,2562.022177,558049.0,NaN,NaN


In [163]:
cdf = df[['Close']] 
cdf 

,Close
Date,
2010-01-01,669.844609
2010-01-04,661.160448
2010-01-05,664.214219
2010-01-06,653.945915
2010-01-07,642.475188
...,...
2018-12-25,2550.461719
2018-12-26,2559.775610
2018-12-27,2562.022177


In [164]:
cdf['prediction_col'] = cdf[['Close']].shift(-1)
cdf

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Close,prediction_col
Date,,
2010-01-01,669.844609,661.160448
2010-01-04,661.160448,664.214219
2010-01-05,664.214219,653.945915
2010-01-06,653.945915,642.475188
2010-01-07,642.475188,631.844249
...,...,...
2018-12-25,2550.461719,2559.775610
2018-12-26,2559.775610,2562.022177
2018-12-27,2562.022177,2543.768822


In [165]:
forecast_out = 30
x = np.array(cdf.drop(['prediction_col'],1))
x = x[:-forecast_out]
x.shape

(2317, 1)

In [166]:
y = np.array(cdf['prediction_col'])
y = y[:-forecast_out]
y

array([ 661.160448,  664.214219,  653.945915, ..., 2523.034883,
       2508.338592, 2496.029279])

In [167]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [168]:
svr_rbf = SVR(kernel='rbf',C=1e3,gamma=0.1)
svr_rbf.fit(x_train,y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [169]:
lr = LinearRegression()
lr.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [170]:
predict_score_svr = svr_rbf.score(x_test,y_test)
print(predict_score_svr)
predict_score_lr = lr.score(x_test,y_test)
print(predict_score_lr)

0.943994965565659
0.9978718929650252


In [171]:
x_forecast = np.array(cdf.drop(['prediction_col'],1))
x_forecast = x_forecast[-forecast_out:]
print(x_forecast.shape)
print(x_forecast)

(30, 1)
[[2496.029279]
 [2438.133381]
 [2424.747587]
 [2424.747587]
 [2480.116097]
 [2433.453034]
 [2437.431329]
 [2549.853274]
 [2570.165982]
 [2561.086108]
 [2587.155643]
 [2587.38966 ]
 [2544.330464]
 [2608.076796]
 [2550.88295 ]
 [2564.455958]
 [2666.300318]
 [2654.786663]
 [2642.664563]
 [2659.3266  ]
 [2659.279797]
 [2679.966932]
 [2702.619814]
 [2628.904342]
 [2550.461719]
 [2550.461719]
 [2559.77561 ]
 [2562.022177]
 [2543.768822]
 [2546.249406]]


In [172]:
svr_rbf_predict = svr_rbf.predict(x_forecast)
print(svr_rbf_predict)
lr_predict = lr.predict(x_forecast)
print(lr_predict)

[2517.85246892 2255.24596193 2504.10826197 2504.10826197 2430.09481175
 2524.94967887 2236.90108053 2530.55307482 2554.7051917  2557.56023893
 2588.4853947  2587.47432388 2529.74610291 2620.10633164 2546.38889608
 2546.07402998 2130.17571653 2618.27341854 2648.15787382 2665.68292945
 2664.98410766 2648.4179742  2711.52779821 2589.312594   2538.26798502
 2538.26798502 2574.00079647 2555.48146585 2531.57267853 2549.54210812]
[2495.74988845 2437.93226196 2424.56456469 2424.56456469 2479.85822004
 2433.25824249 2437.23115909 2549.50111689 2569.78636342 2560.71876483
 2586.7530555  2586.98675612 2543.98577337 2607.64592445 2550.52940084
 2564.08405901 2665.79073202 2654.29264274 2642.18693103 2658.82644203
 2658.77970231 2679.43886964 2702.06112641 2628.44531296 2550.10873931
 2550.10873931 2559.41003853 2561.65356832 2543.42489068 2545.90212109]
